In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import requests
from datetime import datetime, timedelta
import time
import logging
import pytz
import aiohttp
import asyncio

In [ ]:
# ticker symbols from every major index

sp_tickers = ['AAPL',
'NVDA',
'MSFT',
'GOOG',
'GOOGL',
'AMZN',
'META',
'BRK-B',
'TSLA',
'AVGO',
'LLY',
'WMT',
'JPM',
'V',
'UNH',
'XOM',
'ORCL',
'MA',
'PG',
'HD',
'COST',
'JNJ',
'ABBV',
'NFLX',
'BAC',
'KO',
'CRM',
'CVX',
'TMUS',
'MRK',
'AMD',
'PEP',
'CSCO',
'LIN',
'WFC',
'ACN',
'ADBE',
'MCD',
'TMO',
'PM',
'ABT',
'NOW',
'IBM',
'AXP',
'MS',
'GE',
'TXN',
'CAT',
'QCOM',
'ISRG',
'VZ',
'DHR',
'DIS',
'AMGN',
'GS',
'INTU',
'CMCSA',
'NEE',
'RTX',
'PFE',
'T',
'BKNG',
'SPGI',
'AMAT',
'LOW',
'UBER',
'BLK',
'PGR',
'UNP',
'SYK',
'BX',
'HON',
'ETN',
'SCHW',
'LMT',
'TJX',
'COP',
'BSX',
'KKR',
'VRTX',
'C',
'ANET',
'ADP',
'PANW',
'UPS',
'MDT',
'CB',
'NKE',
'FI',
'BMY',
'DE',
'GILD',
'MU',
'ADI',
'SBUX',
'BA',
'MMC',
'PLD',
'AMT',
'SO',
'ELV',
'LRCX',
'MO',
'PLTR',
'HCA',
'INTC',
'MDLZ',
'SHW',
'CI',
'ICE',
'DUK',
'REGN',
'KLAC',
'DELL',
'EQIX',
'WM',
'ABNB',
'WELL',
'CTAS',
'TT',
'GEV',
'MCO',
'CEG',
'GD',
'ZTS',
'CME',
'PH',
'APH',
'AON',
'PYPL',
'SNPS',
'ITW',
'CMG',
'CL',
'CDNS',
'USB',
'PNC',
'MSI',
'MAR',
'NOC',
'TDG',
'CRWD',
'CVS',
'ECL',
'MMM',
'APD',
'TGT',
'EOG',
'BDX',
'FDX',
'ORLY',
'CARR',
'MCK',
'CSX',
'FCX',
'WMB',
'COF',
'RSG',
'AJG',
'EMR',
'ADSK',
'FTNT',
'DLR',
'NXPI',
'AFL',
'PSA',
'ROP',
'HLT',
'TFC',
'NSC',
'OKE',
'SLB',
'TRV',
'GM',
'RCL',
'SPG',
'BK',
'MET',
'DHI',
'PCAR',
'KMI',
'GWW',
'URI',
'SRE',
'AEP',
'O',
'FANG',
'MNST',
'NEM',
'AZO',
'JCI',
'PAYX',
'PSX',
'ALL',
'AMP',
'D',
'CPRT',
'FIS',
'AIG',
'FICO',
'MPC',
'CHTR',
'CCI',
'OXY',
'LHX',
'LEN',
'ROST',
'MSCI',
'CMI',
'COR',
'PWR',
'KDP',
'KMB',
'FAST',
'TEL',
'PEG',
'PRU',
'KVUE',
'PCG',
'ODFL',
'NDAQ',
'VST',
'CTVA',
'AME',
'STZ',
'HES',
'F',
'HWM',
'KHC',
'EW',
'VLO',
'KR',
'GEHC',
'EA',
'OTIS',
'EXC',
'CBRE',
'MCHP',
'IR',
'IT',
'VRSK',
'GLW',
'GRMN',
'IQV',
'DFS',
'GIS',
'LVS',
'ACGL',
'CTSH',
'A',
'BKR',
'XEL',
'DAL',
'YUM',
'IRM',
'MPWR',
'LULU',
'SYY',
'TRGP',
'VMC',
'MLM',
'HSY',
'ED',
'RMD',
'EXR',
'DD',
'DOW',
'HPQ',
'NUE',
'VICI',
'IDXX',
'WAB',
'EFX',
'ETR',
'AVB',
'MTB',
'HIG',
'AXON',
'EIX',
'HUM',
'CNC',
'WTW',
'WEC',
'XYL',
'ROK',
'RJF',
'BRO',
'CSGP',
'FITB',
'ON',
'PPG',
'TSCO',
'CCL',
'TTWO',
'LYB',
'ANSS',
'NVR',
'EBAY',
'K',
'STT',
'VTR',
'MTD',
'DXCM',
'EQR',
'AWK',
'LYV',
'SW',
'GPN',
'ADM',
'PHM',
'DOV',
'CAH',
'UAL',
'TYL',
'KEYS',
'DTE',
'HPE',
'BIIB',
'IFF',
'VLTO',
'CDW',
'DVN',
'FTV',
'SBAC',
'EL',
'DECK',
'CHD',
'LDOS',
'HAL',
'BR',
'TROW',
'FE',
'PPL',
'NTAP',
'ES',
'ERIE',
'GDDY',
'AEE',
'WY',
'HUBB',
'CPAY',
'ROL',
'HBAN',
'WDC',
'WST',
'PTC',
'CBOE',
'CINF',
'EQT',
'WRB',
'STE',
'RF',
'SYF',
'ATO',
'TDY',
'ZBH',
'FSLR',
'BF-B',
'STX',
'MKC',
'TSN',
'CMS',
'COO',
'MRNA',
'PKG',
'STLD',
'EXPE',
'WBD',
'BLDR',
'NTRS',
'CLX',
'OMC',
'INVH',
'ARE',
'ZBRA',
'CNP',
'IP',
'BBY',
'WAT',
'FOXA',
'LH',
'PFG',
'DRI',
'CFG',
'HOLX',
'LUV',
'ESS',
'FOX',
'NRG',
'MOH',
'BALL',
'JBHT',
'BAX',
'MAA',
'GEN',
'CTRA',
'L',
'J',
'DG',
'SNA',
'FDS',
'ULTA',
'DGX',
'TER',
'MAS',
'KEY',
'VRSN',
'EXPD',
'AVY',
'HRL',
'SMCI',
'IEX',
'PNR',
'NI',
'AMCR',
'KIM',
'PODD',
'GPC',
'DOC',
'NWS',
'EG',
'NWSA',
'ALGN',
'LNT',
'AKAM',
'MRO',
'TXT',
'APTV',
'TRMB',
'CF',
'RVTY',
'SWK',
'NDSN',
'INCY',
'DPZ',
'UDR',
'VTRS',
'CPB',
'SWKS',
'CAG',
'EVRG',
'POOL',
'CE',
'JBL',
'DLTR',
'UHS',
'FFIV',
'JKHY',
'REG',
'JNPR',
'BXP',
'SOLV',
'CPT',
'RL',
'EMN',
'HST',
'CHRW',
'ALLE',
'SJM',
'BG',
'TECH',
'AES',
'PAYC',
'DVA',
'NCLH',
'DAY',
'TAP',
'KMX',
'ENPH',
'MGM',
'ALB',
'TPR',
'LW',
'IPG',
'AOS',
'MKTX',
'BEN',
'EPAM',
'CTLT',
'WYNN',
'GNRC',
'AIZ',
'PNW',
'LKQ',
'GL',
'MTCH',
'FRT',
'CRL',
'TFX',
'HAS',
'HSIC',
'APA',
'MOS',
'CZR',
'MHK',
'FMC',
'WBA',
'IVZ',
'HII',
'BWA',
'PARA',
'QRVO',
'BBWI']

dow_tickers = ['AAPL',
'MSFT',
'AMZN',
'WMT',
'JPM',
'V',
'UNH',
'HD',
'PG',
'JNJ',
'KO',
'CRM',
'CVX',
'MRK',
'CSCO',
'MCD',
'IBM',
'AXP',
'CAT',
'VZ',
'DIS',
'AMGN',
'GS',
'HON',
'NKE',
'BA',
'INTC',
'MMM',
'TRV',
'DOW']

nasdaq_tickers = ['AAPL',
'NVDA',
'MSFT',
'GOOG',
'GOOGL',
'AMZN',
'META',
'TSLA',
'AVGO',
'COST',
'NFLX',
'ASML',
'TMUS',
'AMD',
'PEP',
'AZN',
'CSCO',
'LIN',
'ADBE',
'TXN',
'QCOM',
'ISRG',
'AMGN',
'INTU',
'CMCSA',
'PDD',
'BKNG',
'AMAT',
'ARM',
'SNY',
'HON',
'VRTX',
'ADP',
'PANW',
'ADI',
'GILD',
'MU',
'SBUX',
'MELI',
'LRCX',
'INTC',
'MDLZ',
'REGN',
'KLAC',
'EQIX',
'ABNB',
'CTAS',
'CEG',
'CME',
'SNPS',
'PYPL',
'CDNS',
'MAR',
'CRWD',
'MRVL',
'ORLY',
'CSX',
'DASH',
'WDAY',
'ADSK',
'FTNT',
'NXPI',
'TTD',
'JD',
'ROP',
'APP',
'PCAR',
'AEP',
'FANG',
'MNST',
'NTES',
'MSTR',
'PAYX',
'CPRT',
'TEAM',
'COIN',
'CHTR',
'ROST',
'KDP',
'FAST',
'ODFL',
'NDAQ',
'DDOG',
'TCOM',
'KHC',
'GEHC',
'EA',
'EXC',
'MCHP',
'VRSK',
'ACGL',
'CTSH',
'XEL',
'BKR',
'MPWR',
'LULU',
'CCEP',
'ALNY',
'IDXX',
'ARGX',
'AXON',
'BIDU',
'WTW',
'CSGP',
'FITB',
'ON',
'FER',
'TSCO',
'TTWO',
'ERIC',
'ANSS',
'EBAY',
'ZS',
'FCNCO',
'FCNCA',
'DXCM',
'TW',
'BNTX',
'UAL',
'BIIB',
'CDW',
'LI',
'SBAC',
'VOD',
'TROW',
'NTAP',
'ERIE',
'ILMN',
'ZM',
'HBAN',
'WDC',
'PTC',
'CINF',
'BGNE',
'KSPI',
'LPLA',
'HOOD',
'FSLR',
'STX',
'RYAAY',
'COO',
'MRNA',
'STLD',
'EXPE',
'WBD',
'MDB',
'GFS',
'NTRS',
'EXE',
'ZBRA',
'FOXA',
'CHKP',
'PFG',
'FWONK',
'FWONA',
'HOLX',
'FOX',
'JBHT',
'ICLR',
'CG',
'GEN',
'DKNG',
'ULTA',
'SSNC',
'TER',
'LINE',
'VRSN',
'WMG',
'NTNX',
'SMCI',
'IBKR',
'UTHR',
'GRAB',
'PODD',
'MANH',
'NWS',
'ENTG',
'NWSA',
'BSY',
'ALGN',
'LNT',
'AKAM',
'NTRA',
'AZPN',
'TRMB',
'MNDY',
'CASY',
'GMAB',
'NDSN',
'INCY',
'DOCU',
'MORN',
'SWKS',
'VTRS',
'CPB',
'POOL',
'FLEX',
'EVRG',
'GLPI',
'AFRM',
'DLTR',
'Z',
'LAMR',
'FFIV',
'EWBC',
'SMMT',
'FTAI',
'JKHY',
'ARCC',
'ZG',
'PCVX',
'TTEK',
'FUTU',
'REG',
'SFM',
'SAIA',
'TXRH',
'EXAS',
'BMRN',
'DUOL',
'LOGI',
'HST',
'OKTA',
'CHRW',
'NBIX',
'RPRX',
'SRPT',
'SOFI',
'CYBR',
'TECH',
'PPC',
'INSM',
'LBRDA',
'LBRDK',
'CART',
'HTHT',
'PAA',
'NICE',
'ENPH',
'MMYT',
'MBLY',
'ALAB',
'LECO',
'MKTX',
'WYNN',
'PCTY',
'CHDN',
'RIVN',
'ESLT',
'DOX',
'MEDP',
'COKE',
'WWD',
'SEIC',
'RGLD',
'LKQ',
'EXEL',
'MTCH',
'XP',
'WIX',
'HAS',
'AAON',
'TLN',
'SIRI',
'ITCI',
'DSGX',
'BILI',
'GGAL',
'FYBR',
'HSIC',
'AAL',
'RVMD',
'ENSG',
'ALTR',
'VFS',
'ROKU',
'APA',
'GTLB',
'OLED',
'BRKR',
'ROIV',
'AUR',
'CZR',
'DBX',
'FSV',
'CFLT',
'WING',
'LNW',
'LEGN',
'PNFP',
'CBSH',
'MTSI',
'DRS',
'WBA',
'VKTX',
'OTEX',
'AGNC',
'CIGI',
'WTFC',
'CHRD',
'ZION',
'RGEN',
'APPF',
'MASI',
'PARAA',
'HQY',
'LNTH',
'HLNE',
'UFPI',
'ASND',
'PARA',
'LBTYB',
'LBTYK',
'SAIC',
'DJT',
'LBTYA',
'CGNX',
'GNTX',
'TPG',
'CELH',
'MIDD',
'GRFS',
'MAT',
'LSCC',
'BOKF',
'TEM',
'HCP',
'PEGA',
'CVLT',
'QRVO',
'JAZZ',
'EXLS',
'SATS',
'MKSI',
'MRNO',
'FRHC',
'VERX',
'CCCS',
'BPOP',
'FRPT',
'HALO',
'BZ',
'CRDO',
'AMKR',
'GLBE',
'WSC',
'CWST',
'SPSC',
'NUVL',
'CROX',
'QXO',
'LSTR',
'ONB',
'NOVT',
'LFUS',
'IONS',
'COLB',
'AVAV',
'RCM',
'IEP',
'ETSY',
'CYTK',
'CSWI',
'CRUS',
'NSIT',
'LCID',
'BECN',
'MMSI',
'REYN',
'COOP',
'NXST',
'WFRD',
'SRCL',
'NXT',
'OLLI',
'VRNS',
'BPMC',
'SIGI',
'BCPC',
'MDGL',
'UMBF',
'NVMI',
'CHX',
'PI',
'LLYVK',
'ACT',
'VNOM',
'CACC',
'LLYVA',
'LYFT',
'RKLB',
'CRVL',
'FFIN',
'FIVE',
'ACIW',
'RMBS',
'UBSI',
'QFIN',
'CORT',
'ITRI',
'OZK',
'CRNX',
'RNA',
'MARA',
'KRYS',
'EXPO',
'RDNT',
'AVT',
'VLY',
'ASTS',
'TENB',
'OS',
'NVEI',
'STRL',
'ESGR',
'WAY',
'SLM',
'XRAY',
'TIGO',
'LANC',
'TSEM',
'COLM',
'RARE',
'FCFS',
'PECO',
'BGC',
'SBRA',
'ALGM',
'HWC',
'BBIO',
'PRCT',
'ACLX',
'RUSHA',
'RUSHB',
'QLYS',
'FELE',
'SHC',
'UPST',
'EEFT',
'NCNO',
'CALM',
'IMVT',
'LITE',
'VRRM',
'IESC',
'STEP',
'AXSM',
'GDS',
'FIZZ',
'ROAD',
'ICUI',
'ALKS',
'IAC',
'CRSP',
'AEIS',
'WRD',
'ACHC',
'GBDC',
'ZI',
'LOPE',
'VCTR',
'CLBT',
'OPCH',
'IPAR',
'PLXS',
'SITM',
'INTA',
'ALVO',
'SKYW',
'IBOC',
'WEN',
'SANM',
'CCOI',
'FTDR',
'ADMA',
'GLNG',
'BLKB',
'ALKT',
'DNLI',
'SRAD',
'DOOO',
'IDCC',
'BANF',
'SGRY',
'ATAT',
'TFSL',
'NWL',
'ASO',
'AXNX',
'CAMT',
'IBRX',
'FRSH',
'DORM',
'TCBI',
'WDFC',
'TGTX',
'IPGP',
'BL',
'KTOS',
'CORZ',
'MRUS',
'POWI',
'CORZ',
'SMTC',
'SLAB',
'CVCO',
'RELY',
'FOLD',
'PAGP',
'SMPL',
'APLS',
'IRDM',
'FULT',
'ARLP',
'EBC',
'OTTR',
'CATY',
'PCH',
'MGEE',
'URBN',
'NWE',
'FIBK',
'PAYO',
'OMAB',
'SHOO',
'USLM',
'CARG',
'HCM',
'FROG',
'BRZE',
'LIF',
'RUN',
'JJSF',
'ICFI',
'FHB',
'PTON',
'IOVA',
'EWTX',
'XENE',
'NEOG',
'PTCT',
'BWIN',
'NMIH',
'AMED',
'RIOT',
'POWL',
'RRR',
'CAR',
'FORM',
'PTEN',
'APGE',
'PENN',
'RYTM',
'ZLAB',
'WSFS',
'SFNC',
'KYMR',
'PLTK',
'SYM',
'BHF',
'MLCO',
'LOT',
'ACVA',
'NARI',
'PATK',
'SNEX',
'INTR',
'MLTX',
'DYN',
'MQ',
'ACLS',
'JANX',
'FLNC',
'WAFD',
'CNXC',
'MGRC',
'LIVN',
'PRGS',
'AGYS',
'TMDX',
'PYCR',
'CLSK',
'CVBF',
'VRNA',
'SYNA',
'GH',
'DIOD',
'PTGX',
'IOSP',
'INDB',
'FA',
'HUBG',
'MEOH',
'ASTH',
'ALRM',
'GSHD',
'VIRT',
'LAUR',
'AMRX',
'SRRK',
'BTSG',
'VCYT',
'QDEL',
'HTLF',
'CPRX',
'APPN',
'NMRK',
'LFST',
'AY',
'AGIO',
'RPD',
'VC',
'BATRA',
'IQ',
'TRMD',
'PSMT',
'DLO',
'PPBI',
'AMPH',
'GERN',
'ARCB',
'FFBC',
'WULF',
'DRVN',
'BATRK',
'TBBK',
'TOWN',
'KLIC',
'IBTX',
'PSNY',
'ACAD',
'TWST',
'ARWR',
'PLUS',
'IDYA',
'PLMR',
'CGON',
'LGIH',
'GDRX',
'CAKE',
'TRUP',
'HWKN',
'AMBA',
'LBPH',
'GPCR',
'WERN',
'SBCF',
'CENT',
'GT',
'ALHC',
'TTMI',
'AVPT',
'SBLK',
'WGS',
'VEON',
'IRTC',
'SWTX',
'PSEC',
'FIVN',
'BANR',
'ADUS',
'VERA',
'OSIS',
'FLYW',
'TRIP',
'OMCL',
'PRVA',
'WB',
'FRME',
'ODD',
'KROS',
'VCEL',
'NFE',
'TRMK',
'JAMF',
'PCT',
'VSEC',
'SLNO',
'STRA',
'GOGL',
'NBTB',
'AKRO',
'MYRG',
'WSBC',
'WVE',
'HLMN',
'SPNS',
'UFPT',
'ZD',
'VICR',
'TFIN',
'ROCK',
'VIAV',
'RNW',
'PTVE',
'TNDM',
'EXPI',
'CENTA',
'MYGN',
'JBLU',
'CLOV',
'AZTA',
'ROIC',
'EFSC',
'RXST',
'MRCY',
'EXTR',
'PINC',
'LION',
'LGND',
'LMAT',
'EVCM',
'TXG',
'YY',
'LILA',
'SYBT',
'CNTA',
'MRX',
'ENLT',
'NMRA',
'LILAK',
'MNKD',
'SUPN',
'HURN',
'HEES',
'DNUT',
'BLTE',
'IAS',
'GLPG',
'HRMY',
'PDCO',
'CRTO',
'OSW',
'MIRM',
'SOUN',
'CLBK',
'DSGR',
'SKWD',
'RXRX',
'CLMT',
'IREN',
'ARVN',
'UPWK',
'PAX',
'SIMO',
'CASH',
'BEAM',
'MGNI',
'PLUG',
'FBNC',
'AVDX',
'NRIX',
'CMPR',
'CHCO',
'PZZA',
'NWBI',
'CLDX',
'NEO',
'MBIN',
'TARS',
'LKFN',
'CIFR',
'VRDN',
'COCO',
'CNXN',
'SYRE',
'NVCR',
'BCRX',
'OCUL',
'APOG',
'CENX',
'BCYC',
'CERT',
'STER',
'RUM',
'ATRC',
'HPK',
'VECO',
'GTX',
'SNDX',
'UPBD',
'KNSA',
'OPRA',
'INFN',
'CHEF',
'MLKN',
'NAMS',
'NAVI',
'ANDE',
'DVAX',
'HUT',
'HROW',
'IMCR',
'RCKT',
'SASR',
'NVAX',
'FOXF',
'SONO',
'HOPE',
'NTCT',
'SGML',
'AFYA',
'SPT',
'FTRE',
'VITL',
'ENVX',
'UCTT',
'NVEE',
'AVDL',
'APLD',
'NEXT',
'TLRY',
'VBTX',
'DAWN',
'COMM',
'PLAY',
'XNCR',
'PRDO',
'SRCE',
'STBA',
'NN',
'IART',
'INMD',
'HELE',
'PLAB',
'NTLA',
'MTTR',
'MCRI',
'TWKS',
'TCBK',
'CRCT',
'AMWD',
'BLMN',
'NSSC',
'STAA',
'BUSE',
'SPRY',
'EVO',
'WABC',
'GO',
'DCBO',
'ULCC',
'TVTX',
'ARDX',
'PGNY',
'BLBD',
'QCRH',
'AHCO',
'IRON',
'ELVN',
'WBTN',
'ADEA',
'RBCAA',
'FORTY',
'UPB',
'ROOT',
'WINA',
'CSGS',
'OCSL',
'VRNT',
'PHVS',
'GIII',
'HLIT',
'THRM',
'BCAX',
'MRTN',
'KURA',
'SSRM',
'COGT',
'RDFN',
'SDGR',
'SABR',
'KARO',
'LZ',
'UNIT',
'BELFA',
'DMLP',
'OPEN',
'MFIC',
'GYRE',
'MOMO',
'PRAX',
'VSAT',
'ESTA',
'NMFC',
'CMPO',
'INVA',
'OLPX',
'GABC',
'SAVA',
'GDYN',
'UDMY',
'IMKTA',
'TIGR',
'ANGI',
'CRAI',
'KALU',
'WOOF',
'DCOM',
'CDNA',
'QNST',
'SEZL',
'ARHS',
'INDV',
'WLFC',
'ACMR',
'SAFT',
'COHU',
'BTDR',
'SBGI',
'PHAT',
'ANIP',
'CSWC',
'ATSG',
'PFBC',
'AMPL',
'MXL',
'ALGT',
'KRUS',
'TBLA',
'TASK',
'COLL',
'MSEX',
'PDFS',
'PNTG',
'TRS',
'PLYA',
'MGPI',
'NRDS',
'XCH',
'PWP',
'ECPG',
'MRVI',
'BLFS',
'PEBO',
'ATEC',
'CCEC',
'XPEL',
'ULH',
'REAX',
'KRNT',
'OCFC',
'EH',
'ASTL',
'NYAX',
'HDL',
'IMTX',
'CBRL',
'DGII',
'AUPH',
'OPK',
'RZLV',
'ODP',
'AMSF',
'ERII',
'VIR',
'SCSC',
'TILE',
'EOLS',
'APLT',
'MDXG',
'FWRG',
'HEPS',
'AMAL',
'SDA',
'FWRD',
'BELFB',
'IGMS',
'BRKL',
'XRX',
'ADV',
'FSUN',
'ARRY',
'IGIC',
'HIMX',
'SBSI',
'MESO',
'AVBP',
'ORKA',
'BITF',
'PLSE',
'SEDG',
'BHRB',
'AUTL',
'AMSC',
'XMTR',
'ARQT',
'ACDC',
'BBSI',
'GCT',
'CSTL',
'AOSL',
'PRTA',
'SEPN',
'JACK',
'PAHC',
'CTBI',
'CMCO',
'CNOB',
'ZYME',
'CAPR',
'BFC',
'SCVL',
'FMBH',
'JBSS',
'CBLL',
'SHLS',
'CSIQ',
'RLAY',
'CRGX',
'CGEM',
'AVAH',
'ICHR',
'LQDA',
'RAPP',
'BMBL',
'AMRK',
'TRNS',
'MBUU',
'PFC',
'ZBIO',
'RDWR',
'FLX',
'CCB',
'BVS',
'NESR',
'VNET',
'PLRX',
'HSTM',
'DHC',
'FIP',
'HTZ',
'PGY',
'CGBD',
'LMB',
'BJRI',
'HTLD',
'TYRA',
'BIOA',
'GPRE',
'CFFN',
'AVO',
'EVGO',
'GOGO',
'ETNB',
'SLRC',
'CECO',
'GDEN',
'TTGT',
'BASE',
'MNRO',
'EYE',
'REPL',
'DNTH',
'NBBK',
'UVSP',
'SLN',
'EMBC',
'IMOS',
'PRAA',
'SANA',
'CRON',
'EGBN',
'PENG',
'PTLO',
'ABCL',
'HCSG',
'HSII',
'MNTK',
'INNV',
'VMEO',
'HFWA',
'CFB',
'ANNX',
'GSM',
'DXPE',
'HAIN',
'ARKO',
'GLDD',
'FCBC',
'BFST',
'TREE',
'UXIN',
'TIPT',
'PCRX',
'MCBS',
'HAYN',
'SHEN',
'FUFU',
'STKL',
'TH',
'OSBC',
'ERAS',
'LAB',
'SWIM',
'SCWX',
'NBN',
'LENZ',
'ADPT',
'MBX',
'PUBM',
'ORRF',
'EU',
'TRIN',
'ASTE',
'ABUS',
'ADSE',
'SNCY',
'KELYB',
'KELYA',
'MATW',
'SPTN',
'HBNC',
'OCS',
'RPAY',
'SBC',
'LXRX',
'STGW',
'MBWM',
'SCHL',
'HAFC',
'IBCP',
'IMNM',
'TCPC',
'DDI',
'NCMI',
'SMBC',
'HBT',
'FTEL',
'CRESY',
'PPTA',
'GOOD',
'HCKT',
'GSBC',
'OPT',
'EOSE',
'AIOT',
'CCAP',
'DJCO',
'DOGZ',
'CRSR',
'EVER',
'ABVX',
'LPRO',
'ANAB',
'ORIC',
'WLDN',
'YMAB',
'DMRC',
'FSBC',
'LQDT',
'FLGT',
'TRML',
'NNE',
'CTLP',
'IRWD',
'OLMA',
'GDEV',
'MLYS',
'AAOI',
'CTKB',
'FDUS',
'STOK',
'CDMO',
'WRLD',
'ATXS',
'ABL',
'KIDS',
'OSPN',
'MREO',
'CLNE',
'TRST',
'IPX',
'CVAC',
'EZPW',
'KC',
'ALMS',
'EXAI',
'IRMD',
'RBBN',
'FNKO',
'CAC',
'AXGN',
'EYPT',
'MLAB',
'ATRO',
'MOFG',
'OFIX',
'ECX',
'RVNC',
'DAKT',
'HUMA',
'TECX',
'ATEX',
'LASR',
'LEGH',
'TRDA',
'HTBK',
'PLPC',
'NTGR',
'CCBG',
'INDI',
'ACIC',
'CNDT',
'SVRA',
'WASH',
'TNGX',
'PHAR',
'SWBI',
'PACB',
'HTBI',
'TERN',
'CRMD',
'WEST',
'AVXL',
'THRD',
'BTBT',
'INOD',
'PGC',
'IMXI',
'CASS',
'CELC',
'HSAI',
'SIBN',
'FFWM',
'GLUE',
'HIFS',
'SPFI',
'QTTB',
'CEVA',
'SNDL',
'FRPH',
'SLP',
'NGNE',
'ALLO',
'RSVR',
'MGIC',
'GEVO',
'ATLC',
'CRVS',
'IIIV',
'CNSL',
'RXT',
'NEXN',
'SEAT',
'BAND',
'VINP',
'VREX',
'MSBI',
'KRT',
'SLRN',
'SVC',
'SRDX',
'LRMR',
'METC',
'CCNE',
'ITRN',
'OFLX',
'NYMT',
'KRRO',
'TALK',
'MPB',
'ESQ',
'METCB',
'GLAD',
'URGN',
'FMNB',
'LX',
'GCMG',
'DGICA',
'CLFD',
'SSYS',
'HIVE',
'SIGA',
'DGICB',
'THFF',
'THRY',
'NFBK',
'YORW',
'UFCS',
'LIND',
'FLWS',
'QTRX',
'AQST',
'LESL',
'HONE',
'ASPI',
'GAIN',
'CRML',
'AURA',
'DAVE',
'PRME',
'LE',
'LUNR',
'ADTN',
'SHBI',
'GCBC',
'OTLY',
'DH',
'ARCT',
'VERV',
'PRTC',
'AROW',
'RITR',
'XERS',
'GLRE',
'NNDM',
'TWFG',
'OABI',
'PFIS',
'ALEC',
'BLDP',
'HCAT',
'CVGW',
'LAND',
'ALT',
'LYTS',
'RNAC',
'SKYT',
'VALN',
'CLMB',
'LVRO',
'CGNT',
'LMNR',
'BCAL',
'FFIC',
'MNMD',
'CGC',
'RDUS',
'VALU',
'LOVE',
'INV',
'ACTG',
'SOHU',
'ABSI',
'NVTS',
'TCBX',
'SHYF',
'LSAK',
'ZVRA',
'ITIC',
'ANSC',
'KE',
'IONR',
'KRNY',
'DADA',
'GRVY',
'NWPX',
'MGTX',
'HNRG',
'SERV',
'MERC',
'KALV',
'CARE',
'RGNX',
'CTNM',
'VLGEA',
'SENEB',
'SENEA',
'NRC',
'BIOX',
'CCSI',
'GRAL',
'FHTX',
'GHRS',
'FDMT',
'DHIL',
'RDVT',
'CBNK',
'LINC',
'PRTH',
'GRPN',
'AEHR',
'BIGC',
'TBPH',
'USAP',
'NETD',
'ESPR',
'RMR',
'WALD',
'BSRR',
'MITK',
'TVGN',
'ALRS',
'ZEUS',
'BWB',
'BSVN',
'RBB',
'ZUMZ',
'NIPG',
'SFIX',
'ENGN',
'EBTC',
'LAZR',
'BYND',
'SGMO',
'VBNK',
'PERI',
'UNTY',
'LWLG',
'RWAY',
'LWAY',
'RICK',
'CCRN',
'HBCP',
'BRY',
'SAGE',
'CWCO',
'LSEA',
'CCCC',
'ZIMV',
'PKOH',
'FISI',
'MXCT',
'ALTI',
'ORGO',
'TKNO',
'AMLX',
'LOCO',
'BWMN',
'LIEN',
'HNST',
'FMAO',
'WTBA',
'HRZN',
'CCIX',
'NRIM',
'VYGR',
'TSHA',
'PRQR',
'NVEC',
'GPAT',
'FRBA',
'TRUE',
'SDST',
'ALF',
'GIG',
'NYXH',
'LGTY',
'MBAV',
'AMCX',
'CDRO',
'DCGO',
'ACNB',
'RRBI',
'BMRC',
'AKBA',
'CWBC',
'FBIZ',
'GAMB',
'TCMD',
'UROY',
'NATH',
'BMEA',
'MTLS',
'CAN',
'PGHL',
'ARTNA',
'JAKK',
'HDSN',
'XOMA',
'FARO',
'CLPT',
'FSBW',
'EWCZ',
'DOYU',
'EVLV',
'LNZA',
'NEWT',
'JFIN',
'NAUT',
'IPXX',
'VMD',
'DENN',
'AIRS',
'CYRX',
'NNOX',
'HEAR',
'PTSI',
'APPS',
'ISPR',
'CIVB',
'JOUT',
'AIRJ',
'NODK',
'CRMT',
'JSPR',
'CVRX',
'ONEW',
'KMDA',
'ITOS',
'BYRN',
'WEYS',
'CMPS',
'SNBR',
'SPOK',
'VACH',
'TMC',
'TITN',
'GMGI',
'ATNI',
'SFST',
'BLZE',
'ALDX',
'ZJYL',
'REAL',
'GTI',
'SSBK',
'ACB',
'DOMO',
'MTRX',
'ITI',
'NECB',
'CERS',
'OBT',
'DSGN',
'CUB',
'IBEX',
'GWRS',
'FLXS',
'INBK',
'OSUR',
'IHRT',
'PANL',
'RZLT',
'FDBC',
'JMSB',
'BEAG',
'PFMT',
'BBCP',
'DCTH',
'REFI',
'TTSH',
'COFS',
'GILT',
'SIMA',
'FNLC',
'ALNT',
'POLE',
'CZNC',
'VCIC',
'ESEA',
'MCFT',
'ALDF',
'MAMA',
'FLIC',
'TMCI',
'ZJK',
'GNFT',
'HOND',
'ZURA',
'USCB',
'PSNL',
'EGHT',
'QURE',
'FORR',
'AFRI',
'SSP',
'CZFS',
'PBFS',
'BLDE',
'ACIU',
'ASLE',
'STRO',
'FRST',
'BTOC',
'PNRG',
'SMTI',
'LPBB',
'LPAA',
'WSBF',
'INZY',
'CCIR',
'ZYXI',
'AUDC',
'WHF',
'AVIR',
'BCML',
'ABEO',
'RGP',
'ANGO',
'VNDA',
'IMMP',
'IMMR',
'POET',
'APEI',
'FATE',
'CBAN',
'RAIL',
'OCGN',
'CDXC',
'PCB',
'AIP',
'SERA',
'UHG',
'RCEL',
'FRGT',
'HRTX',
'IRBT',
'PLL',
'ASUR',
'LNKB',
'ACCD',
'AVNW',
'TLS',
'PDLB',
'CLYM',
'API',
'RGTI',
'SVII',
'PCYO',
'INSE',
'SGHT',
'TTEC',
'LXEO',
'AEYE',
'ANIK',
'PKBK',
'PLBC',
'SGC',
'RENE',
'ARTV',
'ACRV',
'MVBF',
'HLXB',
'PMTS',
'LUNG',
'ICG',
'HURA',
'AMRN',
'BETR',
'BPRN',
'SBT',
'TCRX',
'ARQ',
'ILPT',
'EDIT',
'ENTA',
'RIGL',
'ATYR',
'NWTN',
'TSBK',
'FVCB',
'IVA',
'NATR',
'QRTEB',
'BDSX',
'OMER',
'JRVR',
'NEGG',
'INBX',
'PGEN',
'NWFL',
'PRCH',
'SOPH',
'CHSN',
'HITI',
'LYEL',
'TSVT',
'VIRC',
'MBCN',
'EPIX',
'NVX',
'TROO',
'PSTX',
'MRSN',
'CHMG',
'MODV',
'OKUR',
'PEPG',
'LCNB',
'RCAT',
'SNFCA',
'OACC',
'CMPX',
'OVLY',
'UTMD',
'NKTR',
'MGNX',
'PAL',
'PBPB',
'GLXG',
'FINW',
'MVIS',
'BWFG',
'XBIT',
'NKTX',
'OPBK',
'CDXS',
'ETON',
'SFHG',
'TIL',
'TRVI',
'MOLN',
'HPAI',
'IVCB',
'MACI',
'SLDP',
'PYXS',
'SLDB',
'SCWO',
'PESI',
'PROC',
'DRUG',
'BLFY',
'LFCR',
'VSTA',
'AENT',
'CDLX',
'VABK',
'QRTEA',
'OB',
'MFIN',
'KYTX',
'TCX',
'TUSK',
'KVAC',
'RGCO',
'SMLR',
'CAMP',
'PROK',
'CRNT',
'TZOO',
'SCPH',
'GASS',
'BCBP',
'INGN',
'PWOD',
'FSTR',
'BNTC',
'ISTR',
'FUNC',
'ZNTL',
'FFNW',
'SWKH',
'CDZI',
'BOOM',
'GPRO',
'NBTX',
'BLNK',
'AFCG',
'SANG',
'GHIX',
'ORGN',
'KOD',
'AILE',
'CPSS',
'OBIO',
'TATT',
'DSY',
'RELL',
'INSG',
'CFFI',
'FULC',
'SKIN',
'FCEL',
'GOSS',
'NKLA',
'KLTR',
'SHIP',
'MDWD',
'EML',
'PAYS',
'HWBK',
'BROG',
'PROP',
'CRBP',
'OGI',
'DSP',
'ESOA',
'SEVN',
'ALCO',
'HQI',
'ADAP',
'IVCA',
'EM',
'BSII',
'STRS',
'DMAC',
'NKSH',
'OXSQ',
'CLLS',
'NPCE',
'BZUN',
'FSFG',
'PROF',
'ILLR',
'ELDN',
'MYFW',
'TBRG',
'TDTH',
'ESCA',
'HFFG',
'ESSA',
'RMBL',
'GLSI',
'FCCO',
'SMID',
'BAER',
'WNEB',
'ATAI',
'ARAY',
'ORIS',
'RILY',
'FLL',
'ABOS',
'BVFL',
'CRBU',
'GAUZ',
'PLCE',
'SKGR',
'TSAT',
'MYPS',
'CABA',
'HCVI',
'ATOS',
'MAMO',
'PVBC',
'LFMD',
'VOXX',
'HMST',
'WCT',
'VXRT',
'SPKL',
'CADL',
'NIU',
'BRKH',
'HOFT',
'STHO',
'PTMN',
'MRCC',
'RCMT',
'LOGC',
'SAMG',
'ACTU',
'BWAY',
'AMLI',
'GNSS',
'RTC',
'SGMT',
'JYNT',
'EXFY',
'SVCO',
'DIBS',
'MCAA',
'WILC',
'TWIN',
'IKT',
'YIBO',
'ALCY',
'ACHV',
'RDCM',
'TPIC',
'LGO',
'CLAR',
'QSG',
'CNTX',
'DRTS',
'IBAC',
'QRHC',
'BTMD',
'NNBR',
'QMMM',
'MRBK',
'CTRN',
'MHLD',
'DXLG',
'STRT',
'GALT',
'RCKY',
'CKPT',
'BDTX',
'ESHA',
'FLD',
'ATLO',
'ADVM',
'ATLX',
'GBIO',
'VOXR',
'MMLP',
'UBFO',
'LTRX',
'CFBK',
'SKYE',
'RFAI',
'III',
'EUDA',
'IPHA',
'TNYA',
'RMNI',
'VTYX',
'AHG',
'LFVN',
'YXT',
'ARBE',
'BCSA',
'CXDO',
'BFIN',
'VIGL',
'GEOS',
'VSTM',
'TAYD',
'VRA',
'EHTH',
'EGAN',
'BANX',
'ATGL',
'CRDL',
'FTLF',
'CZWI',
'ISRL',
'CRDF',
'FRAF',
'CGEN',
'RPTX',
'ULBI',
'AKYA',
'HNVR',
'PBYI',
'LAKE',
'CBFV',
'LPTX',
'LTBR',
'INO',
'SSTI',
'HURC',
'CAAS',
'PEBK',
'NVCT',
'TOUR',
'CSTE',
'AFBI',
'MRAM',
'MCRB',
'SFBC',
'ECBK',
'OTLK',
'FNWD',
'GUTS',
'ALTO',
'ALLT',
'FXNC',
'CURI',
'GATE',
'ACRS',
'QH',
'EBMT',
'GGR',
'CSLR',
'FGBI',
'UNB',
'AZ',
'POWW',
'NOVV',
'RMBI',
'CRNC',
'INMB',
'DLTH',
'REE',
'SKYX',
'XNET',
'CEP',
'EPSN',
'AMTX',
'GAIA',
'SBFG',
'MNSB',
'AVTX',
'LCUT',
'ISSC',
'BSET',
'BRLS',
'VERI',
'ADAG',
'TELO',
'FCAP',
'SCLX',
'GRWG',
'SHMD',
'CSPI',
'NTIC',
'KPTI',
'TOYO',
'ME',
'PDSB',
'MAXN',
'TLSI',
'FEIM',
'FENC',
'PLAO',
'KRMD',
'KINS',
'ATOM',
'PFIE',
'BRAG',
'DERM',
'APXI',
'KGEI',
'SEER',
'NUTX',
'KRMD',
'TRVG',
'DLHC',
'LARK',
'CRVO',
'QIPT',
'DTI',
'GOCO',
'CFFS',
'ANIX',
'VBFC',
'HOWL',
'OVBC',
'CBRG',
'KNDI',
'NEON',
'PLMJ',
'RMTI',
'UEIC',
'ELUT',
'CHCI',
'IMPP',
'PCSC',
'AXTI',
'ALLK',
'AOUT',
'LEE',
'IMUX',
'COYA',
'STKS',
'MASS',
'CURR',
'SNCR',
'KOPN',
'TLGY',
'VERU',
'OFS',
'VEEA',
'THCH',
'CMTL',
'ACET',
'EDAP',
'SUUN',
'IVVD',
'CSLM',
'ASMB',
'QUIK',
'CNTY',
'SSSS',
'FBYD',
'LPSN',
'MOND',
'PBHC',
'SFWL',
'PPYA',
'ALVR',
'WMPN',
'RLMD',
'PROV',
'OPAL',
'SRTS',
'USGO',
'OPTN',
'HBIO',
'NEOV',
'GECC',
'QUBT',
'GNLX',
'SPHL',
'QSI',
'OPRT',
'FBLG',
'PPIH',
'IPSC',
'AIRG',
'MPAA',
'BSBK',
'SIFY',
'CBUS',
'PDEX',
'FNGR',
'RVSB',
'SPWH',
'ALAR',
'INVZ',
'OPOF',
'HPH',
'ZBAO',
'THCP',
'ALOT',
'TBMC',
'FONR',
'BOWN',
'RGLS',
'RBKB',
'TLSA',
'PRPL',
'BCOV',
'ASRT',
'CUE',
'CBAT',
'SOWG',
'SRBK',
'PFX',
'MDXH',
'VFF',
'ORMP',
'OPRX',
'MIST',
'CHAR',
'SUNS',
'CODA',
'CVGI',
'AFJK',
'SIEB',
'EVGR',
'GLAC',
'ACAB',
'HUHU',
'ACNT',
'BCAB',
'BZFD',
'REKR',
'AGEN',
'QETA',
'CPHC',
'GNTA',
'BLUE',
'RGC',
'DYCQ',
'BKHA',
'CITE',
'KEQU',
'ONCY',
'SATL',
'IFRX',
'IROH',
'SY',
'RRGB',
'ONYX',
'GIFI',
'MAYS',
'CDTX',
'KVHI',
'VIOT',
'FATBB',
'OPI',
'FAT',
'GAQ',
'FSHP',
'IVAC',
'DTSQ',
'TBNK',
'RENB',
'PWUP',
'FNWB',
'PFTA',
'LGCL',
'BYSI',
'CASI',
'SGA',
'HSPO',
'SND',
'AZI',
'XFOR',
'BYNO',
'HLVX',
'ALRN',
'BIRD',
'EPRX',
'CMRX',
'EMCG',
'CHRS',
'IMAB',
'ARBK',
'CTOR',
'VMCA',
'WW',
'OAKU',
'BLEU',
'VTGN',
'LATG',
'PETS',
'PMVP',
'INVE',
'DPCS',
'WOK',
'WIMI',
'AITR',
'VUZI',
'WAI',
'NPAB',
'BAYA',
'SILC',
'INCR',
'IOBT',
'OVID',
'GAN',
'MNOV',
'WRAP',
'IKNA',
'SLNG',
'ZTEK',
'BRID',
'GSIT',
'PLBY',
'TGAA',
'LBGJ',
'PHUN',
'INAQ',
'RSSS',
'PRLH',
'DBVT',
'MGYR',
'YTRA',
'JVSA',
'SKK',
'LASE',
'ATMV',
'FNVT',
'ASYS',
'HNNA',
'UBCP',
'CTMX',
'ATMC',
'AREC',
'LICN',
'MAPS',
'AUID',
'FVN',
'IVCP',
'BOCN',
'AVTE',
'LSBK',
'CLSD',
'EURK',
'AUBN',
'NCTY',
'SLS',
'MGRM',
'AERT',
'RAPT',
'MSSA',
'MTC',
'FIAC',
'ALXO',
'MCHX',
'KLXE',
'MGX',
'RANI',
'BEEM',
'ANL',
'ECOR',
'REFR',
'MNPR',
'AEAE',
'ELTK',
'ABAT',
'CLRB',
'MODD',
'ELVA',
'FRLA',
'WPRT',
'AIFU',
'GRDI',
'VANI',
'HYPR',
'LILM',
'DIST',
'EHGO',
'NB',
'TYGO',
'ZENV',
'CLEU',
'TETE',
'TELA',
'CTXR',
'QNCX',
'NVAC',
'GLLI',
'VTSI',
'PRLD',
'HAIA',
'ICCH',
'MFH',
'NHTC',
'IROQ',
'SDIG',
'SYRS',
'LRFC',
'STCN',
'FTCI',
'GLE',
'GOEV',
'FUSB',
'NTRB',
'PPSI',
'RDZN',
'ENTX',
'SOTK',
'ARQQ',
'RR',
'KOSS',
'BOLD',
'FOSL',
'BEAT',
'OMGA',
'SCNX',
'USAU',
'LOOP',
'SPRO',
'FTII',
'AIMAU',
'FGL',
'RFAC',
'LVO',
'CRGO',
'CNTB',
'LNSR',
'NSPR',
'KTCC',
'MVST',
'FORA',
'TDUP',
'SHIM',
'FDSB',
'JUNE',
'YI',
'HTLM',
'LOAN',
'PAVS',
'LPTH',
'HGBL',
'LVLU',
'CAPN',
'TOP',
'VRCA',
'GANX',
'MITA',
'STTK',
'CCG',
'UPLD',
'GBBK',
'BYFC',
'UG',
'NA',
'FAAS',
'YHNA',
'TMTC',
'BOTJ',
'SHOT',
'VSTE',
'SELX',
'MURA',
'NVNO',
'BRNS',
'CCTS',
'WFCF',
'PRE',
'MDBH',
'MDIA',
'DTIL',
'KRON',
'ALGS',
'THTX',
'SNYR',
'HYMC',
'GEG',
'SELF',
'CPBI',
'KSCP',
'ONDS',
'XLO',
'DGHI',
'STI',
'HTCO',
'PIII',
'PDYN',
'BAFN',
'SGRP',
'PTHL',
'FOXX',
'BCOW',
'ROCL',
'CTSO',
'VOR',
'FTHM',
'IPWR',
'WLGS',
'IMRX',
'TORO',
'NSTS',
'BRAC',
'UONE',
'BUJA',
'CSBR',
'KZR',
'MTEN',
'SNAL',
'OPXS',
'RGS',
'NDLS',
'LUCD',
'CLRC',
'ATRA',
'RDAC',
'PITA',
'COCH',
'BGM',
'PTLE',
'SCYX',
'UONEK',
'HHS',
'LUNA',
'FBIO',
'AISP',
'AFMD',
'CLST',
'OCX',
'HOUR',
'GRRR',
'TZUP',
'ANGH',
'IDN',
'CPTN',
'DWSN',
'GODN',
'DHAI',
'OPTX',
'EDRY',
'AQU',
'FLNT',
'LINK',
'MARX',
'UNCY',
'WAVE',
'VIVK',
'ICAD',
'CLNN',
'ANEB',
'AIRE',
'FLUX',
'MNY',
'RECT',
'AADI',
'ALSA',
'ELTX',
'TARA',
'FKWL',
'BMR',
'OSS',
'YYGH',
'SHLT',
'LVTX',
'MATH',
'SSBI',
'IZEA',
'PMAX',
'PXS',
'STFS',
'CRWS',
'MSAI',
'RPID',
'INTE',
'OPHC',
'HOOK',
'NCSM',
'GLST',
'NOTV',
'BNIX',
'SKYQ',
'NICK',
'ORKT',
'FORL',
'ASRV',
'BLAC',
'AXDX',
'KACL',
'AIRT',
'IMMX',
'IGTA',
'CAPT',
'EYEN',
'IPW',
'CFSB',
'YOTA',
'DUET',
'LSB',
'TCBS',
'AGAE',
'INTS',
'CREX',
'GENK',
'DXR',
'PRPH',
'ICMB',
'PRTS',
'RFIL',
'CLIR',
'CLGN',
'RAVE',
'XGN',
'AGMH',
'TACT',
'UCL',
'RLYB',
'HSON',
'BWEN',
'SMXT',
'CARM',
'RAND',
'DSWL',
'APYX',
'ACHL',
'SWIN',
'RDIB',
'WAVS',
'FSEA',
'DRRX',
'EBON',
'VCSA',
'PXLW',
'CALC',
'GIFT',
'OKYO',
'NVA',
'JG',
'SYT',
'MNDO',
'NAAS',
'STBX',
'HKIT',
'FARM',
'ESGL',
'CLPS',
'SRZN',
'CENN',
'LEXX',
'WINV',
'HUIZ',
'IXHL',
'CNVS',
'BLRX',
'ICCM',
'ARKR',
'VYNE',
'GDST',
'USEG',
'BHIL',
'VGAS',
'BCG',
'TRSG',
'FEBO',
'CODX',
'HFBL',
'RVPH',
'MCAG',
'USIO',
'IRD',
'ECDA',
'CTRM',
'ADGM',
'PASG',
'JRSH',
'MESA',
'WTMA',
'NXTC',
'BRFH',
'CMBM',
'CCLD',
'BGFV',
'MRKR',
'ABLV',
'KPLT',
'TLF',
'AIXI',
'ISPO',
'MIRA',
'DUOT',
'DARE',
'GLBS',
'LUMO',
'XOS',
'PET',
'AWRE',
'RENT',
'APWC',
'SAG',
'PBBK',
'GTEC',
'SJ',
'TPCS',
'SNT',
'VTVT',
'NTWK',
'ELEV',
'FEAM',
'AAME',
'MYNA',
'OMIC',
'LIVE',
'ASPS',
'GROW',
'LTRN',
'CETY',
'NISN',
'ICLK',
'RCON',
'JYD',
'DFLI',
'DECA',
'TURN',
'ANTX',
'GRCE',
'NSYS',
'SOND',
'RDI',
'XBP',
'MLEC',
'CELU',
'ALTS',
'KALA',
'CDTG',
'FTEK',
'PODC',
'OESX',
'DYAI',
'EQ',
'HYFM',
'INTG',
'DRIO',
'FGEN',
'XTKG',
'SYPR',
'AACG',
'SPAI',
'MIGI',
'STIM',
'LGVN',
'ACXP',
'FGF',
'AKTX',
'SURG',
'BNAI',
'SABS',
'FEMY',
'CHR',
'ELBM',
'GTIM',
'GP',
'OCEA',
'DALN',
'NMTC',
'ITRM',
'ZOOZ',
'JDZG',
'HUDI',
'EVTV',
'STEC',
'LPCN',
'MIND',
'QOMO',
'BNR',
'NAII',
'MDAI',
'ENLV',
'PMEC',
'LRE',
'SLNH',
'SWVL',
'ICCC',
'SAIH',
'OM',
'BTAI',
'SCOR',
'INKT',
'ONMD',
'PMN',
'UTSI',
'CCTG',
'XAIR',
'GDTC',
'BTM',
'FPAY',
'BANL',
'GLYC',
'BCTX',
'GREE',
'RAY',
'NXL',
'MKTW',
'GRYP',
'PLUR',
'BIVI',
'GELS',
'HOLO',
'UCAR',
'CRIS',
'ESLA',
'IRIX',
'EPOW',
'FFIE',
'TNXP',
'AIFF',
'CVKD',
'CLIK',
'ONCO',
'IZM',
'PSHG',
'BOLT',
'TPST',
'GAME',
'DAIO',
'VSEE',
'CPSH',
'BNGO',
'DTST',
'YYAI',
'MOGO',
'NWGL',
'SOHO',
'KFFB',
'NVNI',
'RNXT',
'NRSN',
'AIEV',
'LSTA',
'IINN',
'POCI',
'PYPD',
'UBX',
'HUDA',
'TOI',
'GYRO',
'JL',
'PIRS',
'SHFS',
'ANY',
'WKSP',
'NRBO',
'UGRO',
'CNTM',
'MOVE',
'SPRB',
'APRE',
'CXAI',
'KIRK',
'HTCR',
'NXPL',
'RETO',
'QMCO',
'BRLT',
'XTLB',
'USEA',
'MOBX',
'SWAG',
'CMMB',
'RMCF',
'CYTH',
'CMLS',
'MEIP',
'CVV',
'BTCS',
'ZKIN',
'JZ',
'OCC',
'LFWD',
'AQMS',
'LUCY',
'CGTX',
'ATER',
'ILAG',
'PSIG',
'NXGL',
'JFU',
'BBGI',
'TKLF',
'WETH',
'FCUV',
'FLGC',
'EKSO',
'CGBS',
'COOT',
'JBDI',
'XELB',
'BDMD',
'COCP',
'BIAF',
'ABTS',
'WKHS',
'CPOP',
'BFRG',
'ROMA',
'TOMZ',
'YHGJ',
'OXBR',
'MHUA',
'MSS',
'DUO',
'APCX',
'EDUC',
'LYRA',
'TXMD',
'LOBO',
'BSGM',
'MKDW',
'XHG',
'EMKR',
'LSH',
'CARA',
'JVA',
'SYBX',
'GSIW',
'GORV',
'WVVI',
'TAIT',
'FLYE',
'MRNS',
'BOSC',
'NCI',
'HLP',
'WHLM',
'CPIX',
'TURB',
'GOVX',
'NCNC',
'NERV',
'SDOT',
'PEV',
'ATHA',
'SGMA',
'EZFL',
'MGIH',
'NCRA',
'TRIB',
'EDTK',
'RAYA',
'PALT',
'VRM',
'KZIA',
'HCWB',
'BHAT',
'SEED',
'GLBZ',
'JZXN',
'QTI',
'HWH',
'CJJD',
'GMM',
'SONM',
'GVP',
'FMST',
'PT',
'MBOT',
'TTOO',
'NEPH',
'TLPH',
'SUGP',
'GIGM',
'AMIX',
'MNDR',
'CNFR',
'RMCO',
'JCTC',
'TRAW',
'VISL',
'TENX',
'SLGL',
'EVAX',
'MTEX',
'MTEK',
'CWD',
'GDC',
'WKEY',
'TWG',
'CLRO',
'ICU',
'MMV',
'AWH',
'MRM',
'PMCB',
'NRXP',
'SMSI',
'SVMH',
'IPA',
'COSM',
'PMD',
'ELSE',
'MDRR',
'IMNN',
'SNSE',
'HUBC',
'PETZ',
'APLM',
'MCVT',
'CSCI',
'SSKN',
'LRHC',
'DLPN',
'ARBB',
'CERO',
'EVGN',
'AEI',
'MNTS',
'VRME',
'INTJ',
'CUTR',
'VRAR',
'VVOS',
'SPI',
'INAB',
'BIOR',
'DTSS',
'VCIG',
'XELA',
'OP',
'CISS',
'CING',
'BSLK',
'PXDT',
'INDP',
'CLWT',
'ASTC',
'GVH',
'AYTU',
'ABVE',
'STRR',
'NIXX',
'HRYU',
'IMRN',
'BTOG',
'BLIN',
'BLMZ',
'CJET',
'MBIO',
'LAES',
'BREA',
'EDSA',
'BOF',
'WTO',
'PAVM',
'HTOO',
'FRSX',
'MFI',
'HOFV',
'GFAI',
'KLTO',
'TSBX',
'IPDN',
'KPRX',
'PC',
'DOMH',
'OMEX',
'RVYL',
'BRTX',
'RDHL',
'CMCT',
'GIPR',
'ATHE',
'DRCT',
'MEGL',
'VINC',
'ATIF',
'WISA',
'SOGP',
'AMPG',
'BYU',
'BTBD',
'SIDU',
'SLE',
'LIQT',
'YJ',
'DPRO',
'BCLI',
'NKGN',
'UBXG',
'SNTI',
'YQ',
'EEIQ',
'LIDR',
'TCTM',
'GCTK',
'JAGX',
'SRM',
'BLBX',
'PRPO',
'ZAPP',
'RVSN',
'BFRI',
'FGI',
'AIHS',
'XWEL',
'TRUG',
'CISO',
'HYZN',
'CDT',
'VRAX',
'QNTM',
'TFFP',
'AKTS',
'CARV',
'STRM',
'GNPX',
'MOB',
'ADXN',
'CDIO',
'GDHG',
'SQFT',
'LITM',
'NIVF',
'SNGX',
'HIHO',
'ADD',
'GWAV',
'SCKT',
'CMAX',
'VERB',
'ARTW',
'LEDS',
'ABVC',
'VCNX',
'BCDA',
'UPXI',
'JXJT',
'NURO',
'PWM',
'INHD',
'HCTI',
'KRKR',
'MARPS',
'RNAZ',
'OMH',
'PULM',
'LMFA',
'COEP',
'VIRX',
'ASNS',
'ENSC',
'OTRK',
'EJH',
'GLTO',
'PSTV',
'ULY',
'POLA',
'APM',
'ZEO',
'WAFU',
'WHLR',
'SYRA',
'HAO',
'MBRX',
'ZCMD',
'ANTE',
'CYN',
'ELWS',
'CYCN',
'EFOI',
'APTO',
'NLSP',
'CETX',
'CTNT',
'CRKN',
'ALCE',
'AMST',
'NXTT',
'WBUY',
'GURE',
'LDTC',
'HOVR',
'ALZN',
'KAVL',
'CTCX',
'FTFT',
'ONVO',
'ENG',
'GSUN',
'MRIN',
'GRNQ',
'BGLC',
'XBIO',
'GTBP',
'CREG',
'ATPC',
'LCFY',
'NXU',
'BENF',
'CHNR',
'CHEK',
'PRZO',
'PLRZ',
'CREV',
'RGF',
'MGOL',
'ADIL',
'REBN',
'KITT',
'SPCB',
'BMRA',
'KDLY',
'FBRX',
'OST',
'YGMZ',
'PPBT',
'WINT',
'LDWY',
'SNTG',
'MYNZ',
'PNBK',
'AGRI',
'YOSH',
'JNVR',
'LUXH',
'OCG',
'HOTH',
'ONFO',
'BNRG',
'UPC',
'SGLY',
'INBS',
'LGCB',
'PRTG',
'RKDA',
'NUZE',
'IMTE',
'CMND',
'CNSP',
'ERNA',
'JCSE',
'MGRX',
'VS',
'SBFM',
'DWTX',
'AEMD',
'AYRO',
'BON',
'PRSO',
'CEAD',
'IBG',
'XCUR',
'PEGY',
'CELZ',
'BCAN',
'SANW',
'SILO',
'ADN',
'INTZ',
'NDRA',
'TGL',
'BNZI',
'EVOK',
'DATS',
'LIXT',
'XRTX',
'IMCC',
'AEHL',
'BOXL',
'POAI',
'GV',
'NUKK',
'GLMD',
'ZCAR',
'VEEE',
'PIK',
'SISI',
'AQB',
'SMTK',
'SPPL',
'TIRX',
'ONCT',
'MULN',
'ENVB',
'JTAI',
'LFLY',
'BTCT',
'SHPH',
'VSME',
'PCSA',
'VINO',
'ASTI',
'IDAI',
'SNPX',
'KTTA',
'LKCO',
'AGFY',
'ATNF',
'JWEL',
'ATXG',
'MSGM',
'FORD',
'MDJH',
'WATT',
'LTRY',
'KXIN',
'QNRX',
'IFBD',
'CTHR',
'GENE',
'STSS',
'SYTA',
'LIPO',
'ARTL',
'OBLG',
'LQR',
'HEPA',
'PTPI',
'SNOA',
'NCNA',
'VVPR',
'LXEH',
'INM',
'OLB',
'VLCN',
'DGLY',
'FRES',
'BDRX',
'MLGO',
'IVDA',
'OCTO',
'ATXI',
'LIFW',
'NVOS',
'WLDS',
'AIMD',
'TTNP',
'APVO',
'VRPX',
'BPTH',
'CNET',
'NVFY',
'ALBT',
'BNOX',
'AUUD',
'TCRT',
'TNFA',
'NUWE',
'PIXY',
'SPGC',
'LYT',
'ASST',
'THAR',
'RIME',
'BAOS',
'RELI',
'SOPA',
'FAMI',
'PALI',
'ICON',
'JFBR',
'SCNI',
'NMHI',
'VERO',
'ELAB',
'TAOP',
'MTEM',
'RSLS',
'AREB',
'NAYA',
'SGD',
'SONN',
'SNES',
'XXII',
'EZGO',
'NTRP',
'ISPC',
'BBLG',
'SNAX',
'HSCS',
'PHIO',
'PTIX',
'EAST',
'GRI',
'QLGN',
'WNW',
'SBET',
'VEV',
'BKYI',
'NVVE',
'XYLO',
'SINT',
'SLXN',
'SLRX',
'XPON',
'BJDX',
'XTIA',
'SGBX',
'SVRE',
'ZVSA',
'ICCT',
'ENTO',
'DRMA',
'ALLR',
'SXTC',
'NAOV',
'BACK',
'HSDT',
'AVGR',
'TNON',
'VMAR',
'ACON',
'APDN',
'SOBR',
'WORX',
'NITO',
'TIVC',
'KWE',
'UK',
'GXAI',
'MYSZ',
'CYTO',
'CNEY',
'GNLN',
'AKAN',
'STAF',
'TC',
'SXTP',
'BSFC',
'REVB',
'LGHL',
'CYCC',
'NCPL',
'TANH',
'PBM',
'SMX',
'EDBL',
'IVP',
'DBGI',
'SPRC',
'LGMK',
'TCBP',
'PRFX',
'TRNR',
'STKH',
'ADTX']



In [ ]:
ticker_list = []
for ticker in nasdaq_tickers:
    ticker_list.append(ticker)
    
for ticker in dow_tickers:
    if ticker not in ticker_list:
        ticker_list.append(ticker)
        
for ticker in sp_tickers:
    if ticker not in ticker_list:
        ticker_list.append(ticker)
len(ticker_list)

In [ ]:
# create list of all available historical dates available in Alphavantage

start_date = datetime(2008, 1, 1)
end_date = datetime.now()

date_list = []
curr_date = start_date

while curr_date <= end_date:
    date_list.append(curr_date.date())
    curr_date += timedelta(days=1)

In [ ]:
# Set up API key
prem_apikey = 'intentionally left blank...'

# Set up log file with Central Time
central_tz = pytz.timezone('America/Chicago')

def central_time(*args):
    return datetime.now(central_tz).timetuple()

log_file = 'intentionally_left_blank.../Investing/options_data_pipeline.log'
logging.basicConfig(
    filename=log_file,
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.Formatter.converter = central_time

# Set the maximum number of API calls per minute
MAX_API_CALLS = 75

async def fetch_data(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            return await response.json()
        else:
            logging.error(f"HTTP Error: {response.status} for URL: {url}")
            return None

async def gather_data():
    async with aiohttp.ClientSession() as session:
        with sqlite3.connect('intentionally_left_blank.../Investing/historical_options_data.db') as conn:
            logging.info('Option data pipeline started')

            for ticker in ticker_list:
                data_list = []
                num_calls = 0
                start_time = time.time()

                for date in date_list:
                    # Enforce API rate limit
                    if num_calls >= MAX_API_CALLS:
                        elapsed_time = time.time() - start_time
                        if elapsed_time < 60:
                            await asyncio.sleep(60 - elapsed_time)
                        num_calls = 0
                        start_time = time.time()

                    # Prepare API URL
                    url = f'https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol={ticker}&date={date}&apikey={prem_apikey}'
                    data = await fetch_data(session, url)
                    num_calls += 1

                    if data and data.get('message') == 'success':
                        df = pd.DataFrame(data['data'])
                        df = df[['contractID', 'symbol', 'expiration', 'strike', 'type', 'last', 
                                 'bid', 'ask', 'volume', 'date', 'implied_volatility', 'delta', 
                                 'gamma', 'theta', 'vega', 'rho']]

                        if not df.empty:
                            data_list.append(df)
                            logging.info(f'Successfully gathered data for {ticker} on {date}')
                        else:
                            logging.warning(f'No data returned for {ticker} on {date}')
                    elif data:
                        logging.warning(f"API error message for {ticker} on {date}: {data.get('message')}")
                
                # Commit data to SQLite if available
                if data_list:
                    df_concat = pd.concat(data_list, ignore_index=True)
                    num_records = len(df_concat)
                    df_concat.to_sql(ticker, con=conn, if_exists='append', index=False)
                    logging.info(f'Successfully committed {num_records} records for {ticker} to SQL database')
                else:
                    logging.warning(f'No data to commit to SQL for {ticker}')

async def main():
    try:
        await gather_data()
    except Exception as e:
        logging.error(f'An unexpected error occurred: {e}')
    finally:
        logging.info('Option data pipeline complete')

# Run the main async function
await main()
